In [2]:
import pandas as pd
import random
import string
from datetime import datetime

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [4]:
spark = (
        SparkSession.builder.master("local[*]")
        .appName(name="unittest")
        .config("spark.ui.enabled", "false")
        .config(
            "spark.driver.bindAddress",
            "0.0.0.0",
        ).getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 09:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df=pd.read_csv('../Loan_default.csv')

In [6]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,255337,255338,255339,255340,255341,255342,255343,255344,255345,255346
LoanID,I38PQUQS96,HPSK72WA7R,C1OZ6DPJ8Y,V2KKSFM3UN,EY08JDHTZP,A9S62RQ7US,H8GXPAOS71,0HGZQKJ36W,1R0N3LGNRJ,CM9L1GTT2P,...,DSL4O0KAWD,6V8S5IUS63,O6SWO6CBGB,48LOOK4VR1,AKXAXQN7PG,8C6S86ESGC,98R4KDHNND,XQK1UUUNGP,JAO28CPL4H,ZTH91CGL0B
Age,56,69,46,32,60,25,38,56,36,40,...,64,68,51,41,40,19,32,56,42,62
Income,85994,50432,84208,31713,20437,90298,111188,126802,42053,132784,...,73743,21711,69492,61809,129890,37979,51953,84820,85109,22418
LoanAmount,50587,124440,129188,44799,9139,90448,177025,155511,92357,228510,...,140354,168231,122962,119238,116119,210682,189899,208294,60575,18481
CreditScore,520,458,451,743,633,720,429,531,827,480,...,300,352,348,444,701,541,511,597,809,636
MonthsEmployed,80,15,26,0,8,18,80,67,83,114,...,0,78,66,34,38,109,14,70,40,113
NumCreditLines,4,1,3,3,4,2,1,4,1,4,...,2,2,2,2,3,4,2,3,1,2
InterestRate,15.23,4.81,21.17,7.07,6.51,22.72,19.11,8.15,23.94,9.09,...,4.12,9.71,10.83,19.99,9.91,14.11,11.55,5.29,20.9,6.73
LoanTerm,36,60,24,24,48,24,12,60,48,48,...,12,60,48,36,24,12,24,60,48,12
DTIRatio,0.44,0.68,0.31,0.23,0.73,0.1,0.16,0.43,0.2,0.33,...,0.24,0.36,0.27,0.31,0.23,0.85,0.21,0.5,0.44,0.48


In [7]:
def gen_user_id(no_user:int):
    user_ids=[]
    for _ in range(no_user * 8):
        user_ids.append("".join(random.sample(string.ascii_letters,20)))
    return list(set(user_ids))[:no_user]


In [8]:
list_user_ids=gen_user_id(no_user=50000)
list_user_map_df=random.choices(list_user_ids,k=len(df))

In [9]:
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 4, 1)

# Generate a list of 10 random datetime
random_dates = [start_date + (end_date - start_date) * random.random() for _ in range(len(df))]


In [10]:
random_dates=list(map(lambda x:x.strftime('%Y/%m/%d %H:%M:%S'),random_dates))
# random_dates=list(map(lambda x:x.strftime('%Y/%m/%d %H:%M:%S'),random_dates))

In [11]:
# random_dates

In [12]:
df['user_id']=list_user_map_df

In [13]:
df['txn_datetime']=random_dates

In [14]:
df

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default,user_id,txn_datetime
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0,FOeIRQnsLbpjZvurkMwi,2021/11/10 07:17:55
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0,kyHoSmrOEWTzDRlpqBVQ,2022/08/28 02:45:08
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1,znUAgCQIhmGpWofycevH,2022/09/06 16:18:16
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0,PTwkdNvmUazZjcLElrun,2022/10/17 13:17:40
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0,rLTKXCBusQyfAMZpzHoS,2021/01/19 01:27:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,19,37979,210682,541,109,4,14.11,12,0.85,Bachelor's,Full-time,Married,No,No,Other,No,0,muDptexSOYJqzPHiFcLZ,2022/04/11 18:17:09
255343,98R4KDHNND,32,51953,189899,511,14,2,11.55,24,0.21,High School,Part-time,Divorced,No,No,Home,No,1,mewsjcBIpPJFRauTdtNk,2021/09/08 00:19:19
255344,XQK1UUUNGP,56,84820,208294,597,70,3,5.29,60,0.50,High School,Self-employed,Married,Yes,Yes,Auto,Yes,0,SBxlRjhVgndCAtUQuskf,2023/04/27 10:18:58
255345,JAO28CPL4H,42,85109,60575,809,40,1,20.90,48,0.44,High School,Part-time,Single,Yes,Yes,Other,No,0,vyHrhRlCnwjfYkZuBMGm,2023/06/10 17:29:55


In [15]:
df['user_id'].nunique()

49702

In [16]:
df_spark=spark.createDataFrame(df)

In [19]:
df_spark.show()

24/03/21 09:30:50 WARN TaskSetManager: Stage 0 contains a task of very large size (3565 KiB). The maximum recommended task size is 1000 KiB.
24/03/21 09:30:54 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+--------------------+-------------------+
|    LoanID|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|             user_id|       txn_datetime|
+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+--------------------+-------------------+
|I38PQUQS96| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.44| Bachelor's|     Full-time|     Divorced|        Yes|          Yes|      Other|        Yes|      0|FOeIRQnsLbpjZvurkMwi|2021/11/10 07:17:55|
|HPSK72W

In [26]:
df_spark=df_spark.withColumn('txn_datetime',F.to_timestamp('txn_datetime','yyyy/MM/dd HH:mm:ss'))

In [27]:
df_spark.write.mode('overwrite').parquet('../loan_default_gen')

24/03/21 09:35:34 WARN TaskSetManager: Stage 5 contains a task of very large size (3565 KiB). The maximum recommended task size is 1000 KiB.
24/03/21 09:35:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
df.to_parquet('../loan_default_gen.parquet')

0        2021-12-02 16:27:04.638866
1        2021-07-20 11:03:39.922299
2        2022-01-02 23:34:24.479407
3        2022-07-07 16:30:53.647730
4        2021-11-12 16:09:19.044707
                    ...            
255342   2022-05-28 21:14:09.497644
255343   2021-08-25 19:02:57.848533
255344   2022-03-02 03:25:27.094030
255345   2021-03-28 08:24:46.012417
255346   2021-03-06 04:41:28.578614
Name: txn_datetime, Length: 255347, dtype: datetime64[ns]